In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.tree import DecisionTreeClassifier

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
!unzip -qq "/content/gdrive/MyDrive/open (2).zip"

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [6]:
#런타임을 다시 초기화 할것
!pip install pandas==1.5.0
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 40.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


In [7]:
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8

In [5]:
import autogluon.core as ag
import pandas as pd
import torch
import torch.nn as nn
import xgboost
import lightgbm
# 모든 모듈 다시 import
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.models import CatBoostModel, LGBModel, RFModel, XGBoostModel

In [6]:
X_train = train.drop(['ID','전화해지여부'], axis=1)
y_train = train['전화해지여부']

In [7]:
X_train['target'] = y_train

In [8]:
train_data = TabularDataset(X_train)
test_data = TabularDataset(y_train)

FASTAI

In [ ]:
predictor = TabularPredictor(label='target', eval_metric='f1_macro', problem_type='binary').fit(
    train_data=train_data, 
    excluded_model_types=['CAT'],
    num_stack_levels=3,
    num_gpus=1,  # GPU 사용 여부를 지정합니다.
     time_limit=3600*5,
    presets='best_quality')


No path specified. Models will be saved in: "AutogluonModels/ag-20230328_080041/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 18000s
AutoGluon will save models to "AutogluonModels/ag-20230328_080041/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    30200
Train Data Columns: 12
Label Column: target
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11700.03 MB
	Train Data (Original)  Memory Usage: 2.9 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.


In [ ]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.804352,750.665316,6845.471965,0.143068,22.650508,2,True,11
1,LightGBMXT_BAG_L1,0.802905,747.098292,6822.602893,747.098292,6822.602893,1,True,3
2,KNeighborsDist_BAG_L1,0.789090,3.423955,0.218564,3.423955,0.218564,1,True,2
3,XGBoost_BAG_L1,0.786609,154.417156,3330.221997,154.417156,3330.221997,1,True,9
4,LightGBM_BAG_L1,0.783271,190.950499,3299.056477,190.950499,3299.056477,1,True,4
5,LightGBMLarge_BAG_L1,0.782233,226.009706,3040.139226,226.009706,3040.139226,1,True,10
6,ExtraTreesEntr_BAG_L1,0.761757,1.438298,11.854636,1.438298,11.854636,1,True,8
7,ExtraTreesGini_BAG_L1,0.760038,1.937860,5.818179,1.937860,5.818179,1,True,7
8,RandomForestEntr_BAG_L1,0.746734,1.199455,25.358775,1.199455,25.358775,1,True,6
9,RandomForestGini_BAG_L1,0.746190,1.216024,23.642739,1.216024,23.642739,1,True,5


In [ ]:
# 예측에 도움이 될 수 없는 ID 컬럼을 제거합니다.
# x_train은 독립변수이므로 종속변수를 제거합니다.
x_train = train.drop(['ID', '전화해지여부'], axis = 1)

# y_train은 종속변수의 값을 설정해줍니다.
y_train = train['전화해지여부']

# test 데이터에서 예측에 도움이 될 수 없는 ID 컬럼을 제거합니다.
test = test.drop('ID', axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import f1_score